## MNIST 데이터에 대해 이해하고, 파이토치로 소프트맥스 회귀를 구현하기 MNIST 데이터를 분류해보자.

### 1. MNIST 데이터 이해하기

0부터 9까지의 이미지로 이루어진 손글씨 데이터셋이다.  
60,000개의 훈련 데이터와 레이블,  
총 10,000개의 테스트 데이터와 레이블로 구성되어 있다.  
레이블은 0-9까지 총 10개.  
한 이미지당 28 * 28 픽셀.

### 2. 토치비전(torchvision) 소개하기

유명한 데이터셋들, 이미 구현되어져 있는 유명한 모델들, 일반적인 이미지 전처리 도구들을 포함  
자연어 처리를 위해서는 토치텍스트라는 패키지가 있다.

### 3. 분류기 구현을 위한 사전 설정

In [1]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.nn as nn
import matplotlib.pyplot as plt
import random

In [4]:
USE_CUDA = torch.cuda.is_available()
device = torch.device('cuda' if USE_CUDA else 'cpu')
print(f'다음 기기로 학습합니다 : {device}')

다음 기기로 학습합니다 : cpu


In [5]:
# for reproducibility
random.seed(777)
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [6]:
# hyperparameters
training_epochs = 15
batch_size = 100

### 4. MNIST 분류기 구현하기

torchvision.datasets.dsets.MNIST를 사용하여 MNIST 데이터셋을 불러올 수 있다.

In [ ]:
#MNIST dataset

# transform= : 파이토치 텐서로 바꿔준다.
mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

100.0%
100.0%
100.0%
100.0%


In [8]:
# dataset loader
data_loader = DataLoader(dataset=mnist_train,
                         batch_size=batch_size,
                         shuffle=True,
                         drop_last=True)